# 03. 시계열 분석 - ARIMA (Time Series Analysis - ARIMA)

---

## 목차 / Table of Contents
1. 시계열 데이터 생성 / Time Series Data Generation
2. 정상성 검정 / Stationarity Test
3. ACF/PACF 분석 / ACF/PACF Analysis
4. ARIMA 모델 / ARIMA Model
5. 모델 진단 / Model Diagnostics
6. 예측 / Forecasting
7. 자동 ARIMA (auto_arima)

In [ ]:
# ============================================================
# 라이브러리 임포트 / Library Imports
# ============================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# 시계열 분석 / Time series analysis
from statsmodels.tsa.stattools import adfuller, kpss, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.stats.diagnostic import acorr_ljungbox

# 평가 / Evaluation
from sklearn.metrics import mean_absolute_error, mean_squared_error

# 설정 / Settings
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
plt.rcParams['figure.figsize'] = (12, 6)
sns.set_style('whitegrid')

print("✅ 라이브러리 로드 완료 / Libraries loaded")

---
## ARIMA 개념 정리 / ARIMA Concepts

### ARIMA(p, d, q) 파라미터:

| 파라미터 | 의미 | 결정 방법 |
|----------|------|----------|
| **p** | AR(자기회귀) 차수 | PACF에서 절단점 |
| **d** | 차분 차수 | 정상성 테스트 (ADF) |
| **q** | MA(이동평균) 차수 | ACF에서 절단점 |

### 정상성 (Stationarity):
- **정상 시계열**: 평균, 분산이 시간에 따라 일정
- **비정상 시계열**: 추세나 계절성이 있음 → 차분으로 정상화

---
## 1. 시계열 데이터 생성 / Time Series Data Generation

In [ ]:
# ============================================================
# 시계열 토이 데이터 생성 / Create Time Series Toy Data
# ============================================================

def create_timeseries_data(n_samples=365, freq='D', trend=True, seasonality=True, noise_level=5, random_state=42):
    """
    시계열 토이 데이터 생성
    Create time series toy data
    
    Args:
        n_samples: 샘플 수 / Number of samples
        freq: 주기 ('D'=일, 'H'=시간, 'M'=월) / Frequency
        trend: 추세 포함 여부 / Include trend
        seasonality: 계절성 포함 여부 / Include seasonality
        noise_level: 노이즈 수준 / Noise level
    """
    np.random.seed(random_state)
    
    # 날짜 인덱스 / Date index
    dates = pd.date_range(start='2023-01-01', periods=n_samples, freq=freq)
    t = np.arange(n_samples)
    
    # 기본값 / Base value
    base = 100
    
    # 추세 / Trend (선형 증가)
    trend_component = 0.05 * t if trend else 0
    
    # 계절성 / Seasonality (주간 패턴)
    if seasonality:
        seasonal_component = 10 * np.sin(2 * np.pi * t / 7)  # 7일 주기
    else:
        seasonal_component = 0
    
    # 노이즈 / Noise
    noise = np.random.randn(n_samples) * noise_level
    
    # 최종 시계열 / Final time series
    values = base + trend_component + seasonal_component + noise
    
    # Series 생성 / Create Series
    ts = pd.Series(values, index=dates, name='value')
    
    return ts

# 데이터 생성 / Generate data
ts = create_timeseries_data(n_samples=365, trend=True, seasonality=True)

print(f"시계열 데이터: {len(ts)} samples")
print(f"기간: {ts.index[0]} ~ {ts.index[-1]}")
ts.head(10)

In [ ]:
# ============================================================
# 시계열 시각화 / Visualize Time Series
# ============================================================

fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# 원본 시계열 / Original time series
axes[0].plot(ts.index, ts.values)
axes[0].set_title('Original Time Series / 원본 시계열')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Value')

# 이동평균 추가 / Add moving average
ma_7 = ts.rolling(window=7).mean()
ma_30 = ts.rolling(window=30).mean()
axes[1].plot(ts.index, ts.values, alpha=0.5, label='Original')
axes[1].plot(ts.index, ma_7, label='MA(7)', linewidth=2)
axes[1].plot(ts.index, ma_30, label='MA(30)', linewidth=2)
axes[1].set_title('Time Series with Moving Averages / 이동평균 포함')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Value')
axes[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
# ============================================================
# 시계열 분해 / Time Series Decomposition
# ============================================================

# 추세, 계절성, 잔차 분해 / Decompose into trend, seasonality, residual
decomposition = seasonal_decompose(ts, model='additive', period=7)

fig, axes = plt.subplots(4, 1, figsize=(14, 12))

decomposition.observed.plot(ax=axes[0], title='Observed / 원본')
decomposition.trend.plot(ax=axes[1], title='Trend / 추세')
decomposition.seasonal.plot(ax=axes[2], title='Seasonality / 계절성')
decomposition.resid.plot(ax=axes[3], title='Residual / 잔차')

plt.tight_layout()
plt.show()

---
## 2. 정상성 검정 / Stationarity Test

### ADF (Augmented Dickey-Fuller) 검정:
- **귀무가설 (H0)**: 단위근 존재 = 비정상 시계열
- **대립가설 (H1)**: 단위근 없음 = 정상 시계열
- **p-value < 0.05**: 귀무가설 기각 → 정상 시계열

In [ ]:
# ============================================================
# ADF 검정 함수 / ADF Test Function
# ============================================================

def adf_test(series, name='Series'):
    """
    ADF 정상성 검정 수행
    Perform ADF stationarity test
    
    Returns:
        True if stationary (p < 0.05), False otherwise
    """
    result = adfuller(series.dropna())
    
    print(f"\n{'='*50}")
    print(f"📊 ADF 검정 결과: {name}")
    print(f"{'='*50}")
    print(f"ADF Statistic: {result[0]:.4f}")
    print(f"p-value:       {result[1]:.4f}")
    print(f"Critical Values:")
    for key, value in result[4].items():
        print(f"  {key}: {value:.4f}")
    
    if result[1] < 0.05:
        print(f"\n✅ 결론: 정상 시계열 (p < 0.05)")
        print(f"✅ Conclusion: Stationary (p < 0.05)")
        return True
    else:
        print(f"\n❌ 결론: 비정상 시계열 (p >= 0.05) → 차분 필요")
        print(f"❌ Conclusion: Non-stationary (p >= 0.05) → Differencing needed")
        return False

# 원본 시계열 검정 / Test original series
is_stationary = adf_test(ts, '원본 시계열 / Original')

In [ ]:
# ============================================================
# 차분으로 정상화 / Make Stationary with Differencing
# ============================================================

def make_stationary(series, max_diff=2):
    """
    차분을 통해 정상 시계열로 변환
    Make series stationary through differencing
    
    Returns:
        stationary_series, d (차분 횟수)
    """
    d = 0
    current = series.copy()
    
    while d < max_diff:
        result = adfuller(current.dropna())
        if result[1] < 0.05:
            print(f"✅ d={d}에서 정상성 확보 (p={result[1]:.4f})")
            return current, d
        
        print(f"📊 d={d}: p-value={result[1]:.4f} → 차분 진행")
        d += 1
        current = current.diff().dropna()
    
    print(f"⚠️ {max_diff}번 차분 후에도 비정상")
    return current, d

# 정상화 / Make stationary
ts_stationary, d_order = make_stationary(ts)

print(f"\n최종 차분 차수 (d): {d_order}")

In [ ]:
# ============================================================
# 원본 vs 차분 시계열 비교 / Compare Original vs Differenced
# ============================================================

fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# 원본 / Original
axes[0].plot(ts.values)
axes[0].set_title('Original Time Series (Non-stationary) / 원본 시계열 (비정상)')
axes[0].set_ylabel('Value')

# 차분 후 / After differencing
if d_order > 0:
    axes[1].plot(ts_stationary.values)
    axes[1].set_title(f'After {d_order} Differencing (Stationary) / {d_order}차 차분 후 (정상)')
else:
    axes[1].plot(ts.values)
    axes[1].set_title('Already Stationary / 이미 정상')
axes[1].set_ylabel('Value')
axes[1].axhline(0, color='red', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

---
## 3. ACF/PACF 분석 / ACF/PACF Analysis

### p, q 결정 규칙:
- **p (AR 차수)**: PACF에서 급격히 0으로 떨어지는 지점
- **q (MA 차수)**: ACF에서 급격히 0으로 떨어지는 지점

In [ ]:
# ============================================================
# ACF/PACF 플롯 / ACF/PACF Plots
# ============================================================

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ACF (Autocorrelation Function) → q 결정
plot_acf(ts_stationary.dropna(), lags=30, ax=axes[0])
axes[0].set_title('ACF (for q) / 자기상관함수 (q 결정용)')

# PACF (Partial Autocorrelation Function) → p 결정
plot_pacf(ts_stationary.dropna(), lags=30, ax=axes[1], method='ywm')
axes[1].set_title('PACF (for p) / 편자기상관함수 (p 결정용)')

plt.tight_layout()
plt.show()

print("\n📌 ACF/PACF 해석 가이드:")
print("- ACF가 천천히 감소 + PACF가 lag p에서 절단 → AR(p) 모델")
print("- ACF가 lag q에서 절단 + PACF가 천천히 감소 → MA(q) 모델")
print("- 둘 다 천천히 감소 → ARMA(p,q) 모델")

---
## 4. ARIMA 모델 / ARIMA Model

In [ ]:
# ============================================================
# Train/Test 분할 (시계열) / Train/Test Split (Time Series)
# ============================================================

# ⚠️ 시계열은 시간 순서대로 분할! 랜덤 분할 금지!
# ⚠️ Time series must be split by time order! No random splitting!

train_size = int(len(ts) * 0.8)
train = ts[:train_size]
test = ts[train_size:]

print(f"훈련 데이터: {len(train)} samples ({train.index[0]} ~ {train.index[-1]})")
print(f"테스트 데이터: {len(test)} samples ({test.index[0]} ~ {test.index[-1]})")

In [ ]:
# ============================================================
# ARIMA 모델 학습 / Train ARIMA Model
# ============================================================

# ARIMA(p, d, q) 파라미터 설정
# ACF/PACF 분석 결과에 따라 조정
p = 2  # AR 차수 (PACF에서 결정)
d = d_order  # 차분 차수 (ADF 테스트에서 결정)
q = 1  # MA 차수 (ACF에서 결정)

print(f"ARIMA({p}, {d}, {q}) 모델 학습 중...")

# 모델 학습 / Train model
model = ARIMA(train, order=(p, d, q))
model_fit = model.fit()

# 모델 요약 / Model summary
print(model_fit.summary())

In [ ]:
# ============================================================
# 여러 ARIMA 모델 비교 / Compare Multiple ARIMA Models
# ============================================================

def compare_arima_models(train, test, orders):
    """
    여러 ARIMA 모델 비교
    Compare multiple ARIMA models
    
    Args:
        train: 훈련 데이터
        test: 테스트 데이터
        orders: (p, d, q) 튜플 리스트
    """
    results = []
    
    for order in orders:
        try:
            model = ARIMA(train, order=order)
            model_fit = model.fit()
            
            # 예측 / Forecast
            forecast = model_fit.forecast(steps=len(test))
            
            # 평가 / Evaluate
            mae = mean_absolute_error(test, forecast)
            rmse = np.sqrt(mean_squared_error(test, forecast))
            aic = model_fit.aic
            bic = model_fit.bic
            
            results.append({
                'order': order,
                'AIC': aic,
                'BIC': bic,
                'MAE': mae,
                'RMSE': rmse
            })
            print(f"✅ ARIMA{order}: AIC={aic:.2f}, MAE={mae:.4f}")
        except Exception as e:
            print(f"❌ ARIMA{order}: 실패 - {e}")
    
    return pd.DataFrame(results)

# 비교할 모델들 / Models to compare
orders_to_test = [
    (1, d_order, 0),
    (1, d_order, 1),
    (2, d_order, 0),
    (2, d_order, 1),
    (2, d_order, 2),
]

comparison = compare_arima_models(train, test, orders_to_test)
print("\n" + "="*60)
print("📊 모델 비교 결과 / Model Comparison Results")
print("="*60)
print(comparison.sort_values('AIC'))

---
## 5. 모델 진단 / Model Diagnostics

In [ ]:
# ============================================================
# 잔차 분석 / Residual Analysis
# ============================================================

# 잔차 추출 / Extract residuals
residuals = model_fit.resid

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. 잔차 시계열 / Residual time series
axes[0, 0].plot(residuals)
axes[0, 0].set_title('Residuals / 잔차')
axes[0, 0].axhline(0, color='red', linestyle='--')

# 2. 잔차 히스토그램 / Residual histogram
axes[0, 1].hist(residuals, bins=30, edgecolor='black', density=True)
axes[0, 1].set_title('Residual Distribution / 잔차 분포')

# 3. 잔차 ACF / Residual ACF
plot_acf(residuals.dropna(), lags=20, ax=axes[1, 0])
axes[1, 0].set_title('Residual ACF / 잔차 자기상관')

# 4. Q-Q plot
from scipy import stats
stats.probplot(residuals.dropna(), dist="norm", plot=axes[1, 1])
axes[1, 1].set_title('Q-Q Plot')

plt.tight_layout()
plt.show()

In [ ]:
# ============================================================
# Ljung-Box 검정 (잔차 자기상관 검정)
# Ljung-Box Test (Residual Autocorrelation Test)
# ============================================================

# H0: 잔차는 백색잡음 (자기상관 없음)
# H1: 잔차에 자기상관 존재
# p-value > 0.05 → 귀무가설 채택 → 잔차에 자기상관 없음 (좋음)

lb_test = acorr_ljungbox(residuals.dropna(), lags=[10, 20], return_df=True)

print("\n" + "="*50)
print("📊 Ljung-Box 검정 결과 / Ljung-Box Test Results")
print("="*50)
print(lb_test)

if (lb_test['lb_pvalue'] > 0.05).all():
    print("\n✅ 결론: 잔차에 자기상관 없음 (모델 적합)")
    print("✅ Conclusion: No autocorrelation in residuals (good model fit)")
else:
    print("\n⚠️ 결론: 잔차에 자기상관 존재 (모델 개선 필요)")
    print("⚠️ Conclusion: Autocorrelation in residuals (model improvement needed)")

---
## 6. 예측 / Forecasting

In [ ]:
# ============================================================
# 테스트 데이터 예측 / Forecast Test Data
# ============================================================

# 예측 / Forecast
forecast = model_fit.forecast(steps=len(test))
forecast_conf = model_fit.get_forecast(steps=len(test))
conf_int = forecast_conf.conf_int()

# 평가 / Evaluate
mae = mean_absolute_error(test, forecast)
rmse = np.sqrt(mean_squared_error(test, forecast))
mape = np.mean(np.abs((test - forecast) / test)) * 100

print(f"\n{'='*50}")
print(f"📊 예측 성능 / Forecast Performance")
print(f"{'='*50}")
print(f"MAE:  {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAPE: {mape:.2f}%")

In [ ]:
# ============================================================
# 예측 결과 시각화 / Visualize Forecast Results
# ============================================================

plt.figure(figsize=(14, 6))

# 훈련 데이터 / Training data
plt.plot(train.index, train.values, label='Train', color='blue')

# 실제 테스트 데이터 / Actual test data
plt.plot(test.index, test.values, label='Actual', color='green')

# 예측 / Forecast
plt.plot(test.index, forecast, label='Forecast', color='red', linestyle='--')

# 신뢰구간 / Confidence interval
plt.fill_between(test.index, conf_int.iloc[:, 0], conf_int.iloc[:, 1], 
                 color='red', alpha=0.2, label='95% CI')

plt.title(f'ARIMA Forecast (MAE={mae:.4f}, RMSE={rmse:.4f})')
plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# ============================================================
# 미래 예측 / Future Prediction
# ============================================================

# 전체 데이터로 재학습 / Retrain on full data
model_full = ARIMA(ts, order=(p, d, q))
model_full_fit = model_full.fit()

# 미래 30일 예측 / Forecast next 30 days
future_steps = 30
future_forecast = model_full_fit.forecast(steps=future_steps)
future_conf = model_full_fit.get_forecast(steps=future_steps)
future_conf_int = future_conf.conf_int()

# 시각화 / Visualize
plt.figure(figsize=(14, 6))

# 최근 60일 + 미래 30일 / Last 60 days + Future 30 days
plt.plot(ts.index[-60:], ts.values[-60:], label='Historical', color='blue')
plt.plot(future_forecast.index, future_forecast.values, label='Forecast', color='red', linestyle='--')
plt.fill_between(future_conf_int.index, future_conf_int.iloc[:, 0], future_conf_int.iloc[:, 1], 
                 color='red', alpha=0.2, label='95% CI')

plt.axvline(ts.index[-1], color='gray', linestyle='--', alpha=0.5)
plt.title('Future Forecast (30 days) / 미래 예측 (30일)')
plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

---
## 7. 자동 ARIMA (auto_arima)

pmdarima 라이브러리를 사용하면 최적의 (p, d, q)를 자동으로 찾을 수 있습니다.

In [ ]:
# ============================================================
# auto_arima (pmdarima 필요) / auto_arima (requires pmdarima)
# ============================================================

try:
    from pmdarima import auto_arima
    
    print("auto_arima 실행 중... (시간이 걸릴 수 있습니다)")
    print("Running auto_arima... (this may take a while)")
    
    # 자동으로 최적 파라미터 찾기 / Automatically find optimal parameters
    auto_model = auto_arima(
        train,
        start_p=0, max_p=3,
        start_q=0, max_q=3,
        d=None,  # 자동 결정 / Auto-determine
        seasonal=False,
        trace=True,  # 과정 출력 / Print progress
        error_action='ignore',
        suppress_warnings=True,
        stepwise=True
    )
    
    print(f"\n최적 모델 / Best Model: {auto_model.order}")
    print(auto_model.summary())
    
    # 예측 / Forecast
    auto_forecast = auto_model.predict(n_periods=len(test))
    auto_mae = mean_absolute_error(test, auto_forecast)
    print(f"\nauto_arima MAE: {auto_mae:.4f}")
    
except ImportError:
    print("⚠️ pmdarima 미설치. 설치: !pip install pmdarima")
    print("⚠️ pmdarima not installed. Install: !pip install pmdarima")

---
## 요약: ARIMA 분석 체크리스트 / Summary: ARIMA Analysis Checklist

```python
# 1. 시계열 시각화 및 분해
seasonal_decompose(ts, period=7)

# 2. 정상성 검정 (ADF)
adf_test(ts)  # p < 0.05면 정상

# 3. 차분으로 정상화 (d 결정)
ts_diff = ts.diff().dropna()

# 4. ACF/PACF로 p, q 결정
plot_acf(ts_diff)  # q 결정
plot_pacf(ts_diff) # p 결정

# 5. ARIMA 모델 학습
model = ARIMA(train, order=(p, d, q))
model_fit = model.fit()

# 6. 잔차 진단
acorr_ljungbox(residuals)  # p > 0.05면 OK

# 7. 예측
forecast = model_fit.forecast(steps=n)
```

In [ ]:
print("✅ 시계열 ARIMA 노트북 완료 / Time Series ARIMA notebook complete!")
print("\n다음 노트북: 04_anomaly_detection.ipynb")
print("Next notebook: 04_anomaly_detection.ipynb")